<h1 align="center" style="background-color:#616161;color:white">Linear Regression with SVM</h1>

Adapted from: https://github.com/nfmcclure/tensorflow_cookbook/tree/master/04_Support_Vector_Machines/03_Reduction_to_Linear_Regression


<h3 style="background-color:#616161;color:white">0. Setup</h3>

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Input Parameters</div>

In [1]:
# Root path
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/git/EventPrediction" # BA, Linux

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Common Libraries</div>

In [2]:
# Core
import numpy as np
import pandas as pd
from IPython.core.debugger import Tracer    # Used for debugging
import logging

# File and database management
import csv
import os
import sys
import json
import sqlite3
from pathlib import Path

# Date/Time
import datetime
import time
#from datetime import timedelta # Deprecated

# Visualization
import matplotlib.pyplot as plt             # Quick
%matplotlib inline

# Misc
import random

#-------------- Custom Libs -----------------#
os.chdir(root)

# Import the codebase module
fPath = root + "/1_codemodule"
if fPath not in sys.path: sys.path.append(fPath)

# Custom Libs
import coreCode as cc
import lastfmCode as fm

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Page Specific Libraries</div>

In [3]:
# Data science (comment out if not needed)
#from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from sklearn import metrics
from sklearn import preprocessing

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Declare Functions</div>

In [4]:
def getTrainAndTestData():
    con = sqlite3.connect(dbPath)
    c = con.cursor()

    # Get list of UserIDs 
    users = pd.read_sql_query("Select UserID from tblUsers Where tblUsers.TestUser = 0",con)

    fieldList="t, UserID, HrsFrom6pm, isSun,isMon,isTue,isWed,isThu,isFri,isSat,t1,t2,t3,t4,t5,t10,t12hrs,t24hrs,t1wk,t2wks,t3wks,t4wks"
    trainDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    testDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    periodsInAMonth=int(60/periodGranularity)*24*7*4

    totalRows=0
    
    for user in users.itertuples():
        # Get training dataset
        SqlStr="SELECT {} from tblTimeSeriesData where UserID = {}".format(fieldList,user.userID)
        df = pd.read_sql_query(SqlStr, con)
        totalRows += len(df)
    
        # Cut-off 1
        k = random.randint(periodsInAMonth, len(df))
        #Tracer()()  -- for debugging purposes
        testDf = testDf.append(df.iloc[k:k+periodsInAMonth])[df.columns.tolist()]

        tmp = df.drop(df.index[k:k+periodsInAMonth])

        # Cut-off 2
        k = random.randint(periodsInAMonth, len(tmp))
        testDf = testDf.append(tmp.iloc[k:k+periodsInAMonth])[df.columns.tolist()]
        trainDf = trainDf.append(tmp.drop(tmp.index[k:k+periodsInAMonth]))[df.columns.tolist()]

    if len(trainDf)+len(testDf) == totalRows:
        print('Ok')
    else:
        print("Incorrect. Total Rows = {}. TestDf+TrainDf rows = {}+{}={}".format(totalRows,len(testDf),len(trainDf),len(testDf)+len(trainDf)))
        
    return trainDf, testDf

def getHiddenTestUsers(firstNPerc=1.0):
    con = sqlite3.connect(dbPath)
    c = con.cursor()

    # Get list of UserIDs 
    users = pd.read_sql_query("Select UserID from tblUsers Where tblUsers.TestUser = 1",con)

    fieldList="t, PeriodID, UserID, HrsFrom6pm, isSun,isMon,isTue,isWed,isThu,isFri,isSat,t1,t2,t3,t4,t5,t10,t12hrs,t24hrs,t1wk,t2wks,t3wks,t4wks"
    testDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    periodsInAMonth=int(60/periodGranularity)*24*7*4

    totalRows=0
    
    for user in users.itertuples():
        # Get training dataset
        SqlStr="SELECT {} from tblTimeSeriesData where UserID = {}".format(fieldList,user.userID)
    
        df = pd.read_sql_query(SqlStr, con)
        df["PeriodID"] = df["PeriodID"].astype(int)
        df.sort_values(['PeriodID'])
        totalRows += len(df)
        # Caluclate period cutt-off
        cutoff = int(len(df)*firstNPerc)
        testDf = testDf.append(df.iloc[0:cutoff])[df.columns.tolist()]
 
    testDf["PeriodID"] =  testDf["PeriodID"].astype(int)
    testDf["UserID"] =  testDf["UserID"].astype(int)
    testDf.sort_values(['UserID','PeriodID'], inplace=True)
    return testDf


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Load settings</div>

In [5]:
settingsDict =  cc.loadSettings()
dbPath = root + settingsDict['mainDbPath']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']
periodGranularity = int(settingsDict['periodGranularity'])

<h3 style="background-color:#616161;color:white">1. Load data</h3>

In [6]:
trainDf,testDf = getTrainAndTestData()

#trainDf['t'].replace(to_replace='0', value='-1', inplace=True)
#testDf['t'].replace(to_replace='0', value='-1', inplace=True)
xTrain = trainDf.drop(['t','UserID'], 1).values
yTrain = trainDf['t'].values.astype(int)
# Change the 0's to -1
#yTrain = np.array([1 if y==1 else -1 for y in yTrain])
yTrain =yTrain.reshape(len(yTrain),1)

# Test data
xTest= testDf.drop(['t','UserID'], 1).values
yTest = testDf['t'].values.astype(int)
#yTrain_test = np.array([1 if y==1 else -1 for y in yTrain_test])
yTest=yTest.reshape(len(yTest),1)


# One-Hot version
yTrain_onehot = pd.get_dummies(trainDf['t']).values.astype(float)
# One-Hot version
yTest_onehot = pd.get_dummies(testDf['t']).values.astype(float)

Ok


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Confirm dimensions</div>

In [7]:
numOfFeatures = np.shape(xTrain)[1]
np.shape(xTrain),np.shape(yTrain),np.shape(yTrain_onehot)

((1355016, 20), (1355016, 1), (1355016, 2))

In [8]:
np.shape(xTest), np.shape( yTest),np.shape(yTest_onehot)

((82771, 20), (82771, 1), (82771, 2))

<h3 style="background-color:#616161;color:white">3. Model Three: Logistic Regression with RBF Kernel</h3>

In [9]:
# Set parameters
learning_rate = 0.01
training_iteration = 1
display_step = 2

batch_size=2000
    
numOfFeatures=20 # 784 for MNIST
numOfClasses=1
    
# TF graph input
x = tf.placeholder("float", [None, numOfFeatures]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, numOfClasses]) # 0-9 digits recognition => 10 classes
prediction_grid = tf.placeholder(shape=[None, 1], dtype=tf.float32)   # Only needed for a 1d problem - see later on

# Create a model

# Set model weights
W = tf.Variable(tf.zeros([numOfFeatures, numOfClasses]))
#b = tf.Variable(tf.zeros([numOfClasses]))
# Create variables for svm
b = tf.Variable(tf.random_normal(shape=[1,batch_size]))   # *****Should this be 20 , batch_size?*****

# Define RBF Kernel: https://stackoverflow.com/questions/37362258/creating-a-radial-basis-function-kernel-matrix-in-matlab
# Gaussian (RBF) training function
gamma = tf.constant(-50.0)
dist = tf.reshape(tf.reduce_sum(tf.square(x), 1), [-1,1])
sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(x, tf.transpose(x)))), tf.transpose(dist))
RBFKernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))


# ******This is the code for if you are using an input grid to test out lots of x values. Not going to use this for multi-dim
#rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1]) 
#sq_distPred = tf.add(tf.subtract(distPred, tf.multiply(2., tf.matmul(x, tf.transpose(prediction_grid)))), tf.transpose(rB))
# *****Instead we will use the same kernel as above. 

prediction_output = tf.matmul(tf.multiply(tf.transpose(y),b), RBFKernel)
prediction = tf.reshape(prediction_output-tf.reduce_mean(prediction_output),[-1,1])
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction), tf.squeeze(y)), tf.float32))

##### Compute cost function

first_term = tf.reduce_sum(b)   # sum all elements together
bsq = tf.matmul(tf.transpose(b), b)    # b^2
ysq = tf.matmul(y, tf.transpose(y))  # y^2
second_term = tf.reduce_sum(tf.multiply(RBFKernel, tf.multiply(bsq, ysq)))
cost_function = tf.negative(tf.subtract(first_term, second_term))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
print('Ok')

Ok


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Train the model</div>

In [10]:
# Launch the graph
sess = tf.Session()

# Initializing the variables
init = tf.global_variables_initializer()
sess.run(init)

# Training cycle
for iteration in range(training_iteration):
    avg_cost = 0.
    total_batch = int(len(xTrain)/batch_size)

    # Loop over all batches
    for i in range(total_batch):
        batch_xs = xTrain[i*batch_size:(i*batch_size)+batch_size]
        batch_ys = yTrain[i*batch_size:(i*batch_size)+batch_size]                

        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch

    # Display logs per eiteration step
    #if iteration % display_step == 0:
    #    print ("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

print ("Training completed!")


Training completed!


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Test Hidden Periods</div>

In [11]:
predictions=[]
# Testing cycle
total_batch = int(len(xTest)/batch_size)

if np.mod(len(xTest),batch_size) != 0:
    # tf requires consistent inputs so need to pad
    rowsToPad=batch_size-np.mod(len(xTest),batch_size)
    padRows=np.zeros([rowsToPad,xTest.shape[1]])
    xTest = np.append(xTest,padRows,axis=0)
    padRows=np.zeros([rowsToPad,yTest.shape[1]])
    yTest = np.append(yTest,padRows,axis=0)
    total_batch += 1
 
# Loop over all batches
for i in range(total_batch):
    batch_xs = xTest[i*batch_size:(i*batch_size)+batch_size]
    batch_ys = yTest[i*batch_size:(i*batch_size)+batch_size]                

    p=1*sess.run(tf.greater_equal(prediction, 0.5),feed_dict={x: batch_xs,y: batch_ys})
    predictions= np.append(predictions,p)

n=len(xTest)-rowsToPad
xTest=xTest[0:n]
yTest=yTest[0:n]
predictions=predictions[0:n]

print(metrics.classification_report(yTest,predictions))
print(metrics.confusion_matrix(yTest,predictions))
print("* Precision = labelled as x / how many were actually x in the ones that were labelled")
print("* Recall = labelled as x / how many were actually x in the dataset")

             precision    recall  f1-score   support

        0.0       1.00      0.95      0.98     73972
        1.0       0.71      1.00      0.83      8799

avg / total       0.97      0.96      0.96     82771

[[70465  3507]
 [    3  8796]]
* Precision = labelled as x / how many were actually x in the ones that were labelled
* Recall = labelled as x / how many were actually x in the dataset


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Test Hidden Users</div>

In [12]:
# Get data
testDf2 = getHiddenTestUsers(firstNPerc=0.5)  # Get the first half of everyones history
xTest = testDf2.drop(['t','UserID','PeriodID'], 1).values
yTest = testDf2['t'].values.astype(int)
yTest=yTest.reshape(len(yTest),1)

predictions=[]
total_batch = int(len(xTest)/batch_size)

if np.mod(len(xTest),batch_size) != 0:
    # tf requires consistent inputs so need to pad
    rowsToPad=batch_size-np.mod(len(xTest),batch_size)
    padRows=np.zeros([rowsToPad,xTest.shape[1]])
    xTest = np.append(xTest,padRows,axis=0)
    padRows=np.zeros([rowsToPad,yTest.shape[1]])
    yTest = np.append(yTest,padRows,axis=0)
    total_batch += 1
 
# Loop over all batches
for i in range(total_batch):
    batch_xs = xTest[i*batch_size:(i*batch_size)+batch_size]
    batch_ys = yTest[i*batch_size:(i*batch_size)+batch_size]                

    p=1*sess.run(tf.greater_equal(prediction, 0.5),feed_dict={x: batch_xs,y: batch_ys})
    predictions= np.append(predictions,p)

n=len(xTest)-rowsToPad
xTest=xTest[0:n]
yTest=yTest[0:n]
predictions=predictions[0:n]

print(metrics.classification_report(yTest,predictions))
print(metrics.confusion_matrix(yTest,predictions))
print("* Precision = labelled as x / how many were actually x in the ones that were labelled")
print("* Recall = labelled as x / how many were actually x in the dataset")

             precision    recall  f1-score   support

        0.0       1.00      0.96      0.98    212565
        1.0       0.68      1.00      0.81     17617

avg / total       0.98      0.96      0.97    230182

[[204209   8356]
 [     5  17612]]
* Precision = labelled as x / how many were actually x in the ones that were labelled
* Recall = labelled as x / how many were actually x in the dataset


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Notes on RBF</div>

Source: 
https://github.com/nfmcclure/tensorflow_cookbook/blob/master/04_Support_Vector_Machines/04_Working_with_Kernels/04_svm_kernels.ipynb

https://github.com/nfmcclure/tensorflow_cookbook/blob/master/04_Support_Vector_Machines/04_Working_with_Kernels/04_svm_kernels.py


Good resouorces: 
* http://mccormickml.com/2014/02/26/kernel-regression/
* http://www.cc.gatech.edu/~isbell/tutorials/rbf-intro.pdf
* http://perso.telecom-paristech.fr/~clemenco/Projets_ENPC_files/kernel-log-regression-svm-boosting.pdf

Notes:
$$P(y_t == 1) = b + \sum_i w_i \int RBF(t'; t-t_i, sigma_I) dt$$

$$f(x)=b+
\sum^N_{i=1}w_iRBF(x,x_i)$$

Where $w_i$ are the parameters of the linear regression and $t_i$,sigma_i are the parameters of the kernel (which can be optimised jointly or via CV). Then, for example, you could have t_i = [1 hour, 1 day, 1 week] and sigma_d = [10min, 1hour, 12hours]. That way it would give a real-valued score to all tracks played around 1 hour +-10min ago, around 1 day +-1hour ago, etc.

This is a good reference: http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf
$$x = test data$$
$$x_i = train data$$
$$rA = x_i^2$$
$$rB=x^2$$
$$\gamma ||(x_i^2 -(x_i  x)^2 + x^2)||$$

Note following how this is 'the exact same thing as above' or how it relates to the RBF formulas I see in the pdf ref



The whole trick is based on the fact that you want to compute matrix $K_ij = K(x_i, x_j) = f(||x_i - x_j||^2)$ in an efficient manner. Matrix computations are based on dot products, thus multiplications, not on norm of a difference. If you do not want to use loops (and in languages like matlab or R you do not want to) you have to figure out how to express this $||x_i - x_j||^2$ using matrix operations, thus:

$$||x_i - x_j||^2 = <x_i - x_j, x_i - x_j> 
                = <x_i, x_i> - <x_i, x_j> - <x_j, x_i> + <x_j, x_j>
                = ||x_i||^2 - 2<x_i, x_j> + ||x_j||^2$$

and this is exactly what is implemented

First they take square of your data, as $||x_i||^2 = \sum _a x_i_a^2$

nms = $sum(X'.^2)$;

next they use multiplication with vector of ones to compute the sum opertion getting

nms'*ones(1,n)

which is vector of ||x_i||^2's, and analogously vector of ||x_j||^2's is

ones(n,1)*nms

and finally they compose using decomposition I wrote before, thus

-nms'*ones(1,n) -ones(n,1)*nms + 2*X*X')

is just a matrix A_ij = - ||x_i - x_j ||^2

In your case, you want to have division by 2sigma^2, thus just put it under the exp, after taking previous arugment in brackets, like

Ks = exp(-(nms'*ones(1,n) -ones(n,1)*nms + 2*X*X')/(2*sigma^2));

